In [ ]:
# module import

import pprint
import warnings

import gym_trading_env  # noqa
import gymnasium as gym
import numpy as np
import pandas as pd
import torch
from gymnasium.envs.registration import register
from sb3_contrib import RecurrentPPO
from sb3_contrib.ppo_recurrent import MlpLstmPolicy
from stable_baselines3 import DQN, PPO  # noqa
from stable_baselines3.ppo import MlpPolicy  # noqa
from tqdm import TqdmExperimentalWarning

from preprocess import preprocess

register(
    id='DiscretedTradingEnv',
    entry_point='gte:DiscretedTradingEnv',
    disable_env_checker = True
)
register(
    id='MultiDatasetDiscretedTradingEnv',
    entry_point='gte:MultiDatasetDiscretedTradingEnv',
    disable_env_checker = True
)

In [ ]:
def reward_only_position_changed(history):
    prev_position = history[-2]["position"]
    curr_position = history[-1]["position"]
    holding_fee = 0.01
    holding_cost = 0

    index = 1
    index_limit = len(history)

    while index < index_limit and history["position", -index] == prev_position:
        index += 1
        holding_cost -= holding_fee

    if prev_position == curr_position:
        # if curr_position == 0:
        #     return holding_cost
        # else:
        #     return 0
        return 0
    else:
        return (history["portfolio_valuation", -1] / history["portfolio_valuation", -index] - 1) # / sqrt(index)

In [ ]:
# Enviornment
# df = pd.read_pickle("./data/train/month/2022/2022_12_1m.pkl")
# df = preprocess(df)

env = gym.make(
    "MultiDatasetDiscretedTradingEnv",
    # df=df,
    dataset_dir="./data/train/month/**/*.pkl",
    preprocess=preprocess,
    # reward_function=reward_only_position_changed,
    positions=[-10,0,10],
    trading_fees=0.0001,
    borrow_interest_rate=0.00003,
    portfolio_initial_value=100,
    window_size=10,
    # num_envs=3,
)
# env = gym.wrappers.FlattenObservation(env)

In [ ]:
from stable_baselines3.common.env_checker import check_env
check_env(env)

In [ ]:
done, truncated = False, False
observation, info = env.reset()
pprint.pprint(dict(observation=observation, info=info))

In [ ]:
position_index = env.action_space.sample()
observation, reward, done, truncated, info = env.step(position_index)
pprint.pprint(dict(observation=observation, reward=reward, info=info))

In [ ]:
done, truncated = False, False
observation, info = env.reset()
while not done and not truncated:
    position_index = env.action_space.sample()
    observation, reward, done, truncated, info = env.step(position_index)